### Análise dos Dados de Corridas de Táxi em NY - iFood Case

Neste notebook, são respondidas as perguntas do case técnico de Data Architect, utilizando a camada gold dos dados processados no Databricks.

Serão abordadas:
1. A média de valor total recebido em cada mês por todos os yellow táxis da frota.
2. A média de passageiros por hora do dia, apenas para o mês de maio de 2023, considerando todos os táxis da frota.

Os resultados das análises também serão salvos como tabelas na camada gold, facilitando o consumo posterior.

In [0]:
from pyspark.sql.functions import hour

### A média de valor total recebido em cada mês por todos os yellow táxis da frota.

In [0]:
df_gold_mean_total = spark.sql("""
    SELECT
        YEAR(pickup_datetime) AS ano,
        MONTH(pickup_datetime) AS mes,
        ROUND(AVG(total_amount), 2) AS media_total_amount
    FROM silver.yellow_tripdata
    GROUP BY ano, mes
    ORDER BY ano, mes
""")

In [0]:
df_gold_mean_total.display()

ano,mes,media_total_amount
2023,1,27.36
2023,2,27.29
2023,3,28.21
2023,4,28.69
2023,5,29.37


### A média de passageiros por hora do dia, apenas para o mês de maio de 2023, considerando todos os táxis da frota.


In [0]:
df_gold_mean_passenger_hour = spark.sql("""
SELECT
    HOUR(pickup_datetime) AS hora,
    ROUND(AVG(passenger_count), 2) AS media_passageiros
FROM silver.yellow_tripdata
WHERE YEAR(pickup_datetime) = 2023
  AND MONTH(pickup_datetime) = 5
GROUP BY hora
ORDER BY hora
""")


In [0]:
df_gold_mean_passenger_hour.display()

hora,media_passageiros
0,1.43
1,1.44
2,1.46
3,1.45
4,1.41
5,1.28
6,1.26
7,1.28
8,1.3
9,1.31


### Salvando na gold

In [0]:
# Caminhos para a camada gold
gold_base = "/FileStore/tables/gold"
gold_mean_total = f"{gold_base}/yellow_mean_total"
gold_mean_passenger_hour = f"{gold_base}/yellow_mean_passenger_hour"

In [0]:
# Salva os resultados como Delta Lake
df_gold_mean_total.write.format("delta").mode("overwrite").save(gold_mean_total)
df_gold_mean_passenger_hour.write.format("delta").mode("overwrite").save(gold_mean_passenger_hour)


In [0]:
# Remove tabelas anteriores, se existirem
spark.sql("DROP TABLE IF EXISTS gold.yellow_mean_total")
spark.sql("DROP TABLE IF EXISTS gold.yellow_mean_passenger_hour")

# Cria as tabelas no schema gold apontando para os dados Delta Lake
spark.sql(f"""
    CREATE TABLE gold.yellow_mean_total 
    USING DELTA LOCATION '{gold_mean_total}'
""")

spark.sql(f"""
    CREATE TABLE gold.yellow_mean_passenger_hour 
    USING DELTA LOCATION '{gold_mean_passenger_hour}'
""")

Out[17]: DataFrame[]